# Welcome to the impala-go demo notebook

Let's query some open data!

In [1]:
import (
    // "database/sql"    
    "github.com/samber/lo"
    "github.com/jmoiron/sqlx"
    _ "github.com/sclgo/impala-go"
)

// we'll use Impala on the docker network
const dsn = "impala://impalad"

var db = sqlx.MustOpen("impala", dsn)

Let's initialize our open data table - OpenStreetMap places from <https://daylightmap.org/earth/>.

We use [sqlx](https://github.com/jmoiron/sqlx) library to write more consise Go. `impala-go` works just fine with plain `database/sql` too!

In [2]:
import "strings"

const createTable = `
 CREATE EXTERNAL TABLE IF NOT EXISTS "daylight_earth"(
   "geometry_id" string,
   "class" string,
   "subclass" string,
   "metadata" string,
   "original_source_tags" string,
   "names" string,
   "quadkey" string,
   "wkt" string)
 PARTITIONED BY (
   "theme" string)
 STORED AS PARQUET
 LOCATION
   's3a://daylight-openstreetmap/earth/release=v1.58'
 TBLPROPERTIES (
   'has_encrypted_data'='false',
   'parquet.compression'='GZIP');
`

%%
_ = db.MustExec(strings.ReplaceAll(createTable, `"`, "`"))
_ = db.MustExec("alter table `daylight_earth` recover partitions;")

Let's find some large parks!

In [7]:
import (
    "html/template"
    "github.com/janpfeifer/gonb/gonbui"
)

const query = `
WITH parks as (
    SELECT        
        GET_JSON_OBJECT(names, '$.local') as name,
        CAST(GET_JSON_OBJECT("metadata", '$.surface_area_sq_m') AS double) as area
        --,ST_GEOMFROMTEXT(wkt)
    from "daylight_earth"
    WHERE "class" = 'park'
        AND "subclass" = 'national_park'
        AND theme = 'landuse'
    )
SELECT 
    name,
    area / 1000000 as area
FROM parks
WHERE name is not null
and area > 1e10
LIMIT 10
`

type Park struct {
    Name string
    Area float64
}

%%
output := cache.Cache("parks", func() []Park {
    var output []Park    
    lo.Must0(db.Select(&output, strings.ReplaceAll(query, `"`, "`")))
    return output
})

const htmlTableTemplate = `
<table>
    <thead>
        <tr>
            <th>Name</th>
            <th>Area</th>
        </tr>
    </thead>
    <tbody>
{{ range . }}
<tr>
    <td>{{ .Name }}</td>
    <td>{{ printf "%.2f" .Area }} sq. km.</td>
</tr>
{{ end }}
    </tbody>
</table>    
`

var out bytes.Buffer
tpl := lo.Must(template.New("tbl").Parse(htmlTableTemplate))
lo.Must0(tpl.Execute(&out, output))
gonbui.DisplayHtml(out.String())

Name,Area
Chobe National Park,10543.58 sq. km.
Watarru Indigenous Protected Area,16597.98 sq. km.
Nyerere National Park,28970.13 sq. km.
Sirmilik National Park,22056.92 sq. km.
ᑐᕐᓱᔪᒥ ᓄᓇᕐᔪᐊᑉ ᒥᕐᖑᐃᓯᕐᕕᖓ Parc national Tursujuq,25721.38 sq. km.
Kluane / Wrangell-St. Elias / Glacier Bay / Tatshenshini-Alsek,97699.94 sq. km.
Tsavo East National Park,13385.28 sq. km.
Parc National de Bamingui Bangoran,11136.54 sq. km.
Parque Nacional da Cameia,14326.47 sq. km.
Aulavik National Park,12093.81 sq. km.
